In [1]:
%load_ext autoreload
%autoreload 2

#import sys
#sys.path.append('../../')

import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt

from src.data.helper_closed_transactions import read_epex_file, filter_lead_time, extract_transactions
from src.data.welfare import clean_transactions, read_weekly_prices_file, read_NTC_file, NTC_preparation, match_transactions

#set maximum columns display pandas
pd.options.display.max_columns = 50

### Read EPEX file

In [2]:
df=read_epex_file("../data/external/DE Continuous Orders 20191001.csv",
    fast_load=False)

df_filtered=filter_lead_time(df)
pivoted, pivoted_levels=extract_transactions(df_filtered, new_data_type=False)
pivoted_levels.reset_index(inplace=True)
pivoted_levels=clean_transactions(pivoted_levels)
pivoted_levels.shape    

(12067, 24)

### Read Hydro Price, NTC

In [3]:
wp=read_weekly_prices_file("../data/external/Hydro Generation and Price_CH_2019.csv")

In [4]:
# load NTC
NTC = read_NTC_file("../data/external/NTC_DEandCH_2019.csv")
NTC = NTC_preparation(NTC)

/opt/anaconda3/envs/envalpiq/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


### Read and prepare Power Limit

In [5]:
def read_pw_file(path):
    df = pd.read_csv(path,
                     sep=';',
                     decimal=".",
                     parse_dates=['VALUE_TIME'],
                     date_parser=lambda col: pd.to_datetime(col,
                                                            utc=True,
                                                            format='%d.%m.%Y %H:%M',
                                                            )
                     )
    return df

def pw_preparation(pw):
    # manipulate "Power Limit" file to obtain the same structure of NTC
    
    # from hourly to 15 minutes time resolution
    pw_pivoted = pd.pivot_table(pw,
                                 values=['Max. von Generation [MWh]', 'Min. von Generation [MWh]',  'Upscale Potential [MWh]', 'Donwnscale Potential', ],
                                 index=['VALUE_TIME'],
                                 aggfunc=list,
                                 ).apply(pd.Series.explode).sort_index()
    
    pw_resampled=pw_pivoted.resample('15T', convention='start').ffill()
    
    # restore normal range index
    pw_resampled=pw_resampled.reset_index()
    
    # create column start_time and end_time for the optimizer 

    pw_resampled['start_time'] = pw_resampled['VALUE_TIME']
    pw_resampled['end_time'] = pw_resampled['VALUE_TIME'] + pd.to_timedelta(15, unit='minutes')
   
    # create a column to store update capacity
    pw_resampled['Selling Actual value update [MW]'] = pw_resampled['Upscale Potential [MWh]']
    pw_resampled['Pumping Actual value update [MW]'] = pw_resampled['Donwnscale Potential']

    

    return pw_resampled

In [6]:
power_lim=read_pw_file("../data/external/Hydro Generation up- downscale Potential_CH_2019.csv")
power_lim=pw_preparation(power_lim)

### Optimization process

In [7]:
def optimize_selling(NTC, pw, ex_vol, time, instru_type, p_match):

    # check in the first place whether the contract was matchable according to marginal price of hydro considerations
    if p_match:

        multiplier_instr_type = {'Hour': 1, 'Half Hour': 2, 'Quarter Hour': 4}

        # get the corresponding index for NTC and Power DFs
        index_NTC = NTC[(NTC['start_time'] == time) & (NTC['end_time'] == time +
                                                   pd.Timedelta(60/multiplier_instr_type[instru_type], unit='min'))].index
        index_pw = pw[(pw['start_time'] == time)& (pw['end_time'] == time +
                                                   pd.Timedelta(60/multiplier_instr_type[instru_type], unit='min'))].index
    
        # check the difference_selling
        diff_NTC = np.array([NTC['CH to DE_Actual value (MW) update'].iloc[i] -
                         ex_vol*multiplier_instr_type[instru_type] for i in index_NTC])
        diff_pw = np.array([pw['Selling Actual value update [MW]'].iloc[i] -
                         ex_vol*multiplier_instr_type[instru_type] for i in index_pw])
        
        
        # update the value
        if np.product(diff_NTC >= 0) and np.product(diff_pw >= 0):
            NTC['CH to DE_Actual value (MW) update'].iloc[index_NTC] = diff_NTC
            pw['Selling Actual value update [MW]'].iloc[index_pw] = diff_pw

            return 1
        if np.product(diff_NTC < 0) and np.product(diff_pw >= 0):
            NTC['CH to DE_Actual value (MW) update'].iloc[index_NTC] = [
                0]*len(diff_NTC)
            pw['Selling Actual value update [MW]'].iloc[index_pw] = diff_pw
            return 0
        if np.product(diff_NTC >= 0) and np.product(diff_pw < 0):
            NTC['CH to DE_Actual value (MW) update'].iloc[index_NTC] = diff_NTC
            pw['Selling Actual value update [MW]'].iloc[index_pw] = [
                0]*len(diff_pw)
            return 0
        else :
            NTC['CH to DE_Actual value (MW) update'].iloc[index_NTC] = [
                0]*len(diff_NTC)
            pw['Selling Actual value update [MW]'].iloc[index_pw] = [
                0]*len(diff_pw)
            return 0 
    else: 
        return 0 
            


def optimize_pumping(NTC, pw, ex_vol, time, instru_type, p_match):

    # check in the first place whether the contract was matchable according to marginal price of hydro considerations
    if p_match:

        multiplier_instr_type = {'Hour': 1, 'Half Hour': 2, 'Quarter Hour': 4}

        # get the corresponding index for NTC and Power DFs
        index_NTC = NTC[(NTC['start_time'] == time) & (NTC['end_time'] == time +
                                                   pd.Timedelta(60/multiplier_instr_type[instru_type], unit='min'))].index
        index_pw = pw[(pw['start_time'] == time)& (pw['end_time'] == time +
                                                   pd.Timedelta(60/multiplier_instr_type[instru_type], unit='min'))].index
    
        # check the difference_selling
        diff_NTC = np.array([NTC['DE to CH_Actual value (MW) update'].iloc[i] -
                         ex_vol*multiplier_instr_type[instru_type] for i in index_NTC])
        diff_pw = np.array([pw['Pumping Actual value update [MW]'].iloc[i] -
                         ex_vol*multiplier_instr_type[instru_type] for i in index_pw])
        
        
        # update the value
        if np.product(diff_NTC >= 0) and np.product(diff_pw >= 0):
            NTC['DE to CH_Actual value (MW) update'].iloc[index_NTC] = diff_NTC
            pw['Pumping Actual value update [MW]'].iloc[index_pw] = diff_pw

            return 1
        if np.product(diff_NTC < 0) and np.product(diff_pw >= 0):
            NTC['DE to CH_Actual value (MW) update'].iloc[index_NTC] = [
                0]*len(diff_NTC)
            pw['Pumping Actual value update [MW]'].iloc[index_pw] = diff_pw
            return 0
        if np.product(diff_NTC >= 0) and np.product(diff_pw < 0):
            NTC['DE to CH_Actual value (MW) update'].iloc[index_NTC] = diff_NTC
            pw['Pumping Actual value update [MW]'].iloc[index_pw] = [
                0]*len(diff_pw)
            return 0
        else :
            NTC['DE to CH_Actual value (MW) update'].iloc[index_NTC] = [
                0]*len(diff_NTC)
            pw['Pumping Actual value update [MW]'].iloc[index_pw] = [
                0]*len(diff_pw)
            return 0 
    else: 
        return 0 
            

In [11]:
def update_execution_price(binary_outcome_s, binary_outcome_p, marginal_price_s, marginal_price_p, execution_price):

    if binary_outcome_s == 1:

        return marginal_price_s
    else:
        # set pumping price different to execution_price
        if binary_outcome_p == 1: 
        
            return execution_price
            ##return marginal_price_p
        else:

            return execution_price



def match_transactions_both_sides(pivoted_levels, NTC, wp, power_lim):
        
    # sort the pivoted df in ascending way - useful to match pumping
    pivoted_levels_sort = pivoted_levels.sort_values(
        by=['Execution Price'], ascending=True)

   
    # this is the first date of the day, it is then used to find the week for the price
    #change pumping_threshold value to set the percentage of marginal cost for pumping
    
    pumping_threshold=0.7
    date = pivoted_levels['End Validity Date'].iloc[0]
    weekly_price = [wp['Average Weekly Price [Euro/MWh]'][(wp['start_date'] < date) &
                                                          (wp['End Date'] >= date)].item(), pumping_threshold*wp['Average Weekly Price [Euro/MWh]'][(wp['start_date'] < date) &
                                                          (wp['End Date'] >= date)].item()]
    # do not account for contracts lower than weekly price for selling
    # do not account for contracts higher than pumping price for pumping
    pivoted_levels_sort['weekly_hydro_marginal_price_selling'] = weekly_price[0]
    pivoted_levels_sort['weekly_hydro_marginal_price_pumping'] = weekly_price[1]
    
    pivoted_levels_sort['possible_match_selling'] = pivoted_levels_sort['Execution Price'] >= weekly_price[0]
    pivoted_levels_sort['possible_match_pumping'] = pivoted_levels_sort['Execution Price'] <= weekly_price[1]


    #match between possible contracts, NTC and power capacity for pumping
    pivoted_levels_sort['match_binary_outcome_pumping'] = pivoted_levels_sort.apply(lambda x: optimize_pumping(NTC, power_lim,
                                                                                               x['Executed Volume'], 
                                                                                               x['Delivery Start'],
                                                                                               x['Instrument Type'],
                                                                                               x['possible_match_pumping']), axis=1)
    
    # sort the df in a descending way - useful to match selling
    pivoted_levels_sort = pivoted_levels_sort.sort_values(
        by=['Execution Price'], ascending=False)
            
    pivoted_levels_sort['match_binary_outcome_selling'] = pivoted_levels_sort.apply(lambda x: optimize_selling(NTC, power_lim,
                                                                                               x['Executed Volume'],
                                                                                               x['Delivery Start'],
                                                                                               x['Instrument Type'],
                                                                                               x['possible_match_selling']), axis=1)
        


    
   
 
    #add column with updated execution price N.B. marginal 
    pivoted_levels_sort['A posteriori Execution Price'] = pivoted_levels_sort.apply(
        lambda x: update_execution_price(x['match_binary_outcome_selling'], x['match_binary_outcome_pumping'], x['weekly_hydro_marginal_price_selling'], x['weekly_hydro_marginal_price_pumping'], x['Execution Price']), axis=1)
    
    # eventually sort the table in time order
    pivoted_levels_sort.sort_values(by='End Validity Date', inplace=True)
   
    return pivoted_levels_sort

In [12]:
pivoted_levels_sort = match_transactions_both_sides(pivoted_levels, NTC, wp, power_lim)